<div><img style="float: right; width: 120px; vertical-align:middle" src="https://www.upm.es/sfs/Rectorado/Gabinete%20del%20Rector/Logos/EU_Informatica/ETSI%20SIST_INFORM_COLOR.png" alt="ETSISI logo" />


# Perceptrón simple y perceptrón multicapa<a id="top"></a>

<i><small>Última actualización: 2024-03-04</small></i></div>
***

## Introducción

En el aprendizaje profundo , o _deep learning_, los perceptrones y los perceptrones multicapa se erigen como fundamentos esenciales para entender el desarrollo y la implementación de redes neuronales artificiales. Este _notebook_ tiene como objetivo adentrarse en la comprensión teórica y práctica de estas estructuras, desde sus principios básicos hasta su aplicación en problemas complejos de clasificación y regresión.

El perceptrón, introducido por Frank Rosenblatt en 1957, es el bloque constructivo más simple de una red neuronal, diseñado para realizar clasificaciones binarias de manera eficiente. Aunque su utilidad se ve limitada por su incapacidad para resolver problemas no linealmente separables, su estudio sienta las bases para comprender algoritmos más complejos. Por otro lado, el perceptrón multicapa, o red neuronal de varias capas, supera estas limitaciones mediante la incorporación de una o más capas ocultas entre la entrada y la salida, permitiendo la aproximación a cualquier función continua y la solución de problemas no lineales.

## Objetivos

Los objetivos que trataremos de cubrir en este _notebook_ son los siguientes:

1. Introducir los conceptos básicos: Explicar de manera clara y concisa qué es un perceptrón y un perceptrón multicapa, destacando sus diferencias y el papel que juegan dentro del campo del aprendizaje automático,
2. Explorar el funcionamiento: Desglosar los componentes internos de estos modelos, incluyendo las neuronas, pesos, sesgos y funciones de activación, y explicar cómo interactúan durante el proceso de aprendizaje,
3. Implementación desde cero: Guiar al lector a través de la creación de un perceptrón y un perceptrón multicapa utilizando solamente Python puro, con el fin de solidificar la comprensión de su mecánica interna y los algoritmos de entrenamiento,
4. Implementación con Keras: Introducir el uso de Keras, una biblioteca de alto nivel para redes neuronales, para construir y entrenar perceptrones multicapa de manera más eficiente, permitiendo al lector familiarizarse con herramientas modernas de aprendizaje profundo, y
5. Aplicación práctica: Demostrar la aplicación de estos modelos en tareas de clasificación y regresión, proporcionando ejemplos prácticos sobre _datasets_ para que el lector pueda experimentar y observar su rendimiento.

Con estos objetivos en mente, este notebook aspira no solo a educar sino también a inspirar a los lectores a profundizar en el estudio de las redes neuronales y a explorar sus aplicaciones en el mundo real.

## Bibliotecas y configuración

A continuación importaremos las bibliotecas que se utilizarán a lo largo del _notebook_.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

Si salen algunos _warning_ es porque `tensorflow` es así. Seguramente sea una compilación genérica y requiera de una compilación específica para que desaparezcan. Afortunadamente, no suele pasar nada porque salgan estos warning, sobreviviremos.

Configuraremos también algunos parámetros para adecuar la presentación gráfica.

In [ ]:
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (20, 6),'figure.dpi': 64})

***

## Perceptrón simple

El perceptrón simple, también conocido simplemente como perceptrón, es un modelo de aprendizaje automático lineal que se utiliza para clasificaciones binarias. Como decíamos enla introducción, este modelo fue desarrollado en 1957 por Frank Rosenblatt y puede considerarse como la unidad básica de una red neuronal artificial. La idea central detrás del perceptrón es simular el funcionamiento de una neurona biológica, donde recibe múltiples señales de entrada, las procesa y produce una salida única. En términos matemáticos, el perceptrón toma un vector de características de entrada, les aplica pesos correspondientes, suma estos productos y luego aplica una función de activación, típicamente la función escalón, para determinar si la entrada pertenece a una clase o a otra. Llamaremos de ahora en adelante a este proceso **inferencia**.

El funcionamiento del perceptrón se basa en la actualización iterativa de los pesos asociados a cada característica de entrada. Durante el proceso de entrenamiento, el modelo ajusta estos pesos para minimizar el error en sus predicciones. Si la predicción es incorrecta, los pesos se actualizan en función del error cometido, utilizando una tasa de aprendizaje para controlar la magnitud del ajuste. Esta regla de aprendizaje, conocida como regla de aprendizaje del perceptrón o **regla delta**, permite al modelo aprender la frontera de decisión que mejor separa las dos clases. A pesar de su simplicidad y de estar limitado a problemas linealmente separables, el perceptrón sienta las bases para algoritmos más complejos y constituye el primer paso hacia el entendimiento de las redes neuronales multicapa y el aprendizaje profundo.

### Inferencia en el perceptrón

Como hemos indicado, el funcionamiento del perceptrón simple es dar una salida en función de la suma de las entradas ponderadas por sus respectivos pesos. Esta función es la función escalón, que devuelve 1 si la suma es mayor que un umbral $\mathcal{U}$ y 0 en caso contrario. Matemáticamente, la salida $\hat{y}$ de un perceptrón simple se calcula como sigue:

$$
\hat{y} = f_a(X \cdot W) = f_a\left(\sum_{i=0}^{n} x_i w_i + b\right)
$$

La matriz de pesos tendrá tantas filas como componentes tiene el vector de entrada. De hecho, podemos tener varias salidas (neuronas) diferentes, por lo que si tenemos más de una, la matriz $W$ tendrá tantas columnas como valores de salida

Este proceso se conoce como «inferencia». Vamos a implementarlo:

In [ ]:
class Perceptron:
    def __init__(self, n_inputs, n_outputs):
        # W: (filas + 1 (bias), columnas)
        self.W = np.random.uniform(-0.5, 0.5, (n_inputs + 1, n_outputs))

    def activation(self, X):
        # Función de activación: función escalón
        return np.piecewise(X, [X <= 0, 0 < X], [0, 1])
        
    def inference(self, X):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # Añadimos una columna entera de 1s (la entrada del bias)
        X = np.c_[np.ones(X.shape[0]), X]
        # Calculamos la entrada neta
        Z = X @ self.W
        # Aplicamos la función de activación
        return self.activation(Z)

Su funcionamiento sería el siguiente:

In [ ]:
# Tenemos las entradas (3)
X = np.array([1, 2, 3])
# Creamos el modelo, del que obtendremos dos salidas dadas las entradas
model = Perceptron(n_inputs=3, n_outputs=2)
# Inferimos a ver qué sale
ŷ = model.inference(X)
print(ŷ)

Fantástico, ya tenemos un perceptrón. ¿Para qué nos vale? Realmente para nada. Los valores de los pesos se han inicializado aleatoriamente. Si queremos que de verdad de respuestas a un problema tenemos dos opciones:

1. Poner los valores de los pesos nosotros a mano (tedioso, sobre todo si la matriz de pesos es muy grande).
2. Que se pongan automáticamente de acuerdo al problema.

Evidentemente este segundo paso es el que queremos. De hecho, este paso es lo que hace que estos modelos estén dentro del área del «aprendizaje automático». La neurona va a aprender automáticamente a dar buenos resultados (o al menos lo va a intentar).

El esquema de entrenamiento será de **aprendizaje supervisado**, esto es, el modelo aprenderá a partir de un conjunto de datos en el que se encuentran tanto los valores de entrada como los valores de salida esperados a dichas entradas.

El algoritmo que usaremos será la [regla delta](https://en.wikipedia.org/wiki/Delta_rule).

### Regla delta

Disponemos de una implementación para la inferencia de un perceptrón, pero no sabemos los pesos. Para ello, vamos a introducir un algoritmo a través del cual, disponiendo de un conjunto de datos con entradas y salidas esperadas, tratará de encontrar los pesos idóneos para aproximar los valores todo lo posible a los presentados.

La regla delta calcula cómo tienen que variar los pesos actuales en función del error que se ha cometido. Esta variación obedece a las siguientes ecuaciones:

$$
\begin{align}
\Delta W       &= \alpha X^t (y - \hat{y}) \\
\Delta \vec{b} &= \alpha (y - \hat{y})
\end{align}
$$

Siendo:

- $\alpha$: El factor de aprendizaje, generalmente en el intervalo $(0, 1)$
- $X^t$: La traspuesta de las entradas.
- $y$ e $\hat{y}$: Las salidas esperada y real de la red, respectivamente.

Como se puede observar, la cantidad que variarán los pesos son directamente proporcionales al error cometido (i.e. cuanto mayor es el error más varían) y a la entrada (i.e. cuanto mayor es la entrada, más ha influido en el error).

Tras calcular esa variación, basta con sumársela a los pesos para obtener los nuevos, tal y como se expresa a continuación:

$$
\begin{align}
W_{t+1}       &= W_{t}       + \Delta W_{t+1} \\
\vec{b_{t+1}} &= \vec{b_{t}} + \Delta \vec{b_{t}}
\end{align}
$$

Vamos a implementar el cálculo de los errores en un nuevo método.

In [ ]:
class Perceptron:
    def __init__(self, n_inputs, n_outputs):
        self.W = np.random.uniform(-0.5, 0.5, (n_inputs + 1, n_outputs))

    def activation(self, X):
        # Función de activación: función escalón
        return np.piecewise(X, [X <= 0, 0 < X], [0, 1])
        
    def inference(self, X):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # Añadimos una columna entera de 1s (la entrada del bias)
        X = np.c_[np.ones(X.shape[0]), X]
        # Calculamos la entrada neta
        Z = X @ self.W
        # Aplicamos la función de activación
        return self.activation(Z)
    
    def learn(self, X, y, alpha):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # Calculamos cuál es la salida para la entrada suministrada
        ŷ = self.inference(X)
        # Con ella, calculamos el error cometido
        error = y - ŷ
        # Añadimos una columna entera de 1s (la entrada del bias)
        X = np.c_[np.ones(X.shape[0]), X]
        # Con el error y las entradas, calculamos lo que variarán los pesos
        delta_W = alpha * X.T @ error
        # Por último, actualizamos los pesos de nuestra red
        self.W = self.W + delta_W

**NOTA**: Hay un if donde se llama a la función `expand_dims`. Esto es debido a que, cuando el vector de entrada es es un array de una dimensión, cuando se le pide las traspuesta (con `.T`) numpy nos devuelve el mismo vector, sin trasponer, por lo que antes de pedir la traspuesta nos aseguramos de que tiene al menos dos dimensiones. Esto convertiría, por ejemplo, la entrada `[0, 1]` (una dimensión) en la entrada `[[0, 1]]` (dos dimensiones).

Con este cálculo, conseguimos que los pesos se acerquen un poquito hacia valores que producen salidas con menor error. Veamos cómo se usaría:

In [ ]:
# Entradas para la puera OR
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])
# Salidas esperadas para cada una de las entradas
y = np.array([[0],
              [1],
              [1],
              [1]])
# Creamos el modelo, del que obtendremos dos salidas dadas las entradas
model = Perceptron(n_inputs=2, n_outputs=1)
# Veamos sus pesos y salidas correspondientes
print(f'Pesos antes de "learn":\n {model.W}')
print(f'Salida para {X[0]}: {model.inference(X[0])} (esperada {y[0]})')
print(f'Salida para {X[1]}: {model.inference(X[1])} (esperada {y[1]})')
print(f'Salida para {X[2]}: {model.inference(X[2])} (esperada {y[2]})')
print(f'Salida para {X[3]}: {model.inference(X[3])} (esperada {y[3]})')
# Ahora, a aprender de nuestro errores
model.learn(X[0], y[0], alpha=0.5)
model.learn(X[1], y[1], alpha=0.5)
model.learn(X[2], y[2], alpha=0.5)
model.learn(X[3], y[3], alpha=0.5)
print(f'Pesos después de "learn":\n {model.W}')
print(f'Salida para {X[0]}: {model.inference(X[0])} (esperada {y[0]})')
print(f'Salida para {X[1]}: {model.inference(X[1])} (esperada {y[1]})')
print(f'Salida para {X[2]}: {model.inference(X[2])} (esperada {y[2]})')
print(f'Salida para {X[3]}: {model.inference(X[3])} (esperada {y[3]})')

Podemos ver que los pesos han cambiado y, a lo mejor, que las predicciones son más certeras. Un par de apuntes:

- Hemos enseñado los cuatro ejemplos de nuestro conjunto de entrenamiento (sí, se llama conjunto de entrenamiento al conjunto con el que entrenamos, en aprendizaje automático somos así de originales). A esto se le conoce como _epoch_. Los entrenamientos se miden en epochs.
- La implementación que hemos hecho permite trabajar con matrices en la entrada y salida. Esto quiere decir que el anterior _epoch se podría haber implementado así:

In [ ]:
model = Perceptron(n_inputs=2, n_outputs=1)
print(f'Pesos antes de "learn":\n {model.W}')
model.learn(X, y, alpha=0.5)
print(f'Pesos después de "learn":\n {model.W}')

Así completamos un epoch en mucho menos tiempo. De hecho, las tarjetas gráficas hacen las operaciones con matrices extremadamente rápidas, y es por ello por lo que se usan tanto en aprendizaje automático.

Y ahora que las redes saben aprender de sus errores, vamos a escribir el proceso por el cual aprenderán (o lo intentarán) todo el problema. A este proceso se le llama entrenamiento

### Entrenamiento de un perceptrón

El proceso de entrenamiento es realizar _epochs_ de aprendizaje uno detrás de otro hasta que la red haya aprendido lo que queremos que sepa hacer. O al menos hasta que lo haga «suficientemente» bien. ¡O no! porque siempre puede pasar que la red no aprenda.

Aquí vamos a usar como condición de parada un número fijo de _epochs_. Crearemos un nuevo método que llamaremos `train` que entrenará el modelo un número determinado de `epochs` sobre un conjunto de datos de entrenamiento  `X, y` y con un factor de aprendizaje `alpha`.

Aprovecharemos y añadiremos un parámetro opcional, `trace`, que indicará cada cuantos epochs nos muestra información por pantalla. Por el momento nos ceñiremos la medida de exactitud (_accuracy_) que es el número resultados acertados con respecto del total. Vamos allá

In [ ]:
class Perceptron:
    def __init__(self, n_inputs, n_outputs):
        # Matriz de pesos: (filas: entradas, columnas: salidas, +1: bias)
        self.W = np.random.uniform(-0.5, 0.5, (n_inputs + 1, n_outputs))

    def activation(self, X):
        # Función de activación: función escalón
        return np.piecewise(X, [X <= 0, 0 < X], [0, 1])
        
    def inference(self, X):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # Añadimos una columna entera de 1s (la entrada del bias)
        X = np.c_[np.ones(X.shape[0]), X]
        # Calculamos la entrada neta
        Z = X @ self.W
        # Aplicamos la función de activación
        return self.activation(Z)
    
    def learn(self, X, y, alpha):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # Calculamos cuál es la salida para la entrada suministrada
        ŷ = self.inference(X)
        # Con ella, calculamos el error cometido
        error = y - ŷ
        # Añadimos una columna entera de 1s (la entrada del bias)
        X = np.c_[np.ones(X.shape[0]), X]
        # Con el error y las entradas, calculamos lo que variarán los pesos
        delta_W = alpha * X.T @ error
        # Por último, actualizamos los pesos de nuestra red
        self.W = self.W + delta_W
    
    def train(self, X, y, epochs, alpha, trace=1):
        for epoch in range(0, epochs):
            if epoch % trace == 0:
                print(f'Epoch {epoch}: Accuracy: {self.accuracy(X, y)}')
            self.learn(X, y, alpha)
        print(f'End -> {epochs} epochs, accuracy: {self.accuracy(X, y)}')
    
    def accuracy(self, X, y):
        ŷ = self.inference(X)
        return (y == ŷ).mean()

Ahora vamos a probar a entrenar nuestro perceptrón para una puerta AND de tres entradas 

In [ ]:
DATASET_AND = np.array([
    [0, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 1, 1, 0],
    [1, 0, 0, 0],
    [1, 0, 1, 0],
    [1, 1, 0, 0],
    [1, 1, 1, 1],
])
X = DATASET_AND[:, :-1]  # Entradas: Todas las columnas hasta la última
y = DATASET_AND[:, -1:]  # Salidas: Todas las columnas desde la última

model = Perceptron(n_inputs=3, n_outputs=1)
model.train(X, y, 100, 0.1, 10)

Bueno, parece que aprende. Vamos a la principal limitación de este modelo. ¿qué pasaría si intentamos entrenar una puerta XOR?

In [ ]:
DATASET_XOR = np.array([
    [0, 0, 0],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0],
])
X = DATASET_XOR[:, :-1]  # Entradas: Todas las columnas hasta la última
y = DATASET_XOR[:, -1:]  # Salidas: Todas las columnas desde la última

model = Perceptron(n_inputs=2, n_outputs=1)
model.train(X, y, 100, 0.1, 10)

Da igual el número de _epochs_, factor de aprendizaje o parámetros que usemos. La red nunca aprenderá. La explicación es la siguiente:

**Un perceptrón es un [clasificador lineal](https://es.wikipedia.org/wiki/Clasificador_lineal)**, lo que quiere decir que clasifica basándose en la combinación lineal de las entradas. Dicho de otro modo, tiene que haber una recta en el plano (o un plano en un espacio, o ...) que separe los diferentes ejemplos a un lado y a otro.

¿Y esto qué? Bueno, el problema con la puerta XOR se ve muy fácilmente en la siguiente imagen:

<center>
<figure class="image">
    <img src="images/puertas-or-y-xor.png" alt="Representación en el plano de las puertas OR y XOR" />
    <figcaption><em><strong>Figura 1.</strong>Hay problemas muy simples que no son separables linealmente.</em></figcaption>
</figure>
</center>

Para la puerta OR es fácil encontrar una línea que separe los ejemplos cuyo valor es 1 de los ejemplos cuyo valor es 0. Para la puerta XOR es imposible. A estos problemas se los denomina, respectivamente, separables y no separables linealmente. Y el problema es que en el mundo real apenas hay problemas que sean separables linealmente, por lo que un perceptrón simple no suele ser la mejor opción casi nunca.

## Perceptrón multicapa

El perceptrón multicapa (MLP, del inglés _multilayer perceptron_), representa una evolución significativa del perceptrón simple hacia una arquitectura capaz de abordar la complejidad inherente a problemas no linealmente separables. A diferencia de su predecesor, el MLP incorpora una o más capas ocultas entre la capa de entrada y la capa de salida, lo que le permite modelar funciones no lineales y realizar tareas de clasificación y regresión más sofisticadas.

Cada neurona en estas capas ocultas realiza cálculos similares a los del perceptrón simple, pero la introducción de múltiples capas y la aplicación de **funciones de activación no lineales**, como la función sigmoide, ReLU o tanh, en cada neurona, permiten a la red aprender patrones complejos en los datos.

El entrenamiento de un perceptrón multicapa se realiza a través del algoritmo de retropropagación, un método que ajusta los pesos de la red de manera iterativa con el objetivo de minimizar la diferencia entre las salidas predichas y las salidas reales (error). Este proceso involucra el cálculo del gradiente de la función de pérdida respecto a cada peso en la red, utilizando el cálculo diferencial, y la actualización de los pesos en la dirección que reduce el error. Gracias a esta capacidad de ajuste fino y a la flexibilidad arquitectónica, los perceptrones multicapa han encontrado aplicaciones en una amplia gama de campos, desde el reconocimiento de voz y de imágenes hasta la modelización del lenguaje natural y la predicción de series temporales.

### ¿Por qué esas dos diferencias cambian tanto el comportamiento de las redes?

Las dos diferencias (varias capas y función de activación no lineal) trabajan conjuntamente para abordar el problema de la no linealidad.

Comencemos con el perceptrón de una sola capa. Aunque tengamos varias neuronas en esa capa, cada una funcionará independiente de las demás y decisión será lineal. Por tanto, por muchas neuronas que tenga, podremos dividir el espacio de entrada en muchas regiones diferentes, pero los límites de estas regiones estarán limitadas a dichos planos, por lo que podrán clasificar más de dos grupos, pero los grupos deberán ser linealmente separables.

¿Y por qué no basta con añadir más capas? ¿Por qué ese requisito de la no linealidad en las funciones de activación? Bueno, la respuesta es que una combinación lineal de funciones lineales sigue siendo una función lineal. Veámoslo con matrices que intimida más aunque es más sencillo:

Una función lineal $f_a$ tiene siempre una matriz asociada a la que llaaremos $M_a$. Ahora, recordemos la fórmula de la salida $ŷ$ de un perceptrón en función de su entrada $X$ con una activación lineal $f_a$:

$$
ŷ = f_a(X \cdot W)
$$

Si tenemos varias capas, digamos 2, la salida de una capa será la entrada de la siguiente, por lo que la función de salida quedaría como sigue:

$$
\begin{align}
S^{(1)} &= f_a(X \cdot W^{(1)}) \\
S^{(2)} &= f_a(S^{(1)} \cdot W^{(2)}) = ŷ 
\end{align}
$$

Ahora bien, dado que la función de activación $f_a$ es lineal, tiene una matriz asociada, por lo que podemos reemplazar la función con un producto de matrices:


$$
\begin{align}
ŷ &= f_a(S^{(1)} \cdot W^{(2)}) \\
  &= W_a \cdot S^{(1)} \cdot W^{(2)} \\
  &= W_a \cdot W_a \cdot X \cdot W^{(1)} W^{(2)} \\
  &= W'_a \cdot X \cdot W'^{(1)} \\
  &= f_a' (X \cdot W'^{(1)})
\end{align}
$$

Sin funciones de activación no lineales, el perceptrón multicapa no podrá aprender relaciones no lineales de los datos, básicamente tendría las mismas capacidades que una red con una sola capa. Así que si hay una relación no lineal entre la entrada y la salida, o hay interacciones entre las variables, la red no será capaz de aprenderlas.

### Inferencia en el perceptrón multicapa

La inferencia es similar a la del perceptrón simple. Lo único que variará es que, como podremos tener varias capas, la salida de cada capa será la entrada de la siguiente, por lo que lo tendremos que implementar como un bucle de inferencias capa a capa.

In [ ]:
class MultilayerPerceptron:
    def __init__(self, n_inputs, n_hidden, n_outputs):
        # Matriz de pesos capa 1: (filas: entradas, columnas: salidas)
        self.W1 = np.random.uniform(-0.5, 0.5, (n_inputs, n_hidden))
        self.b1 = np.random.uniform(-0.5, 0.5, (1, n_hidden))
        # Matriz de pesos capa 2: (filas: entradas, columnas: salidas)
        self.W2 = np.random.uniform(-0.5, 0.5, (n_hidden, n_outputs))
        self.b2 = np.random.uniform(-0.5, 0.5, (1, n_outputs))

    def activation(self, X):
        # Función de activación: función sigmoidal
        return 1 / (1 + np.exp(-X))
        
    def inference(self, X):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # Calculamos la salida de la capa 1
        S1 = self.activation(X @ self.W1 + self.b1)
        # Calculamos la salida de la capa 2
        S2 = self.activation(S1 @ self.W2 + self.b2)
        return S2

Su funcionamiento sería el siguiente:

In [ ]:
# Tenemos las entradas (3)
X = np.array([1, 2, 3])
# Creamos el modelo, del que obtendremos dos salidas dadas las entradas
model = MultilayerPerceptron(n_inputs=3, n_hidden=2, n_outputs=2)
# Inferimos a ver qué sale
ŷ = model.inference(X)
print(ŷ)

Ya tenemos la inferencia programada. Ahora vamos con el algoritmo a través del que aprende la red. Ya hemos visto que la regla delta no nos vale porque para las capas ocultas no sabemos la salida esperada y, por tanto, no sabemos lo que nos estamos equivocando en ellas. Usaremos un truco, que es usar el error de las capas posteriores para estimar el error de la capa en la que nos encontramos, y usar la derivada de la función de activación para saber hacia qué punto desciende el error.

### Regla delta generalizada (_backpropagation_) y entrenamiento

Consiste en propagar desde la última capa hasta la primera el error existente entre el valor $ŷ$ que devuelve la red para la entrada $X$ y el valor de salida esperado $y$.

Lo primero que hay que hacer es calcular el error cometido en cada una de las capas para actualizar sus correspondientes pesos. La clave aquí es que hay que calcular **primero el error de la última capa**, y luego el error en las capas anteriores a partir del error inmediatamente posterior. Este error se denote como $\vec{\delta^(k)}$ y se define como sigue:

$$
\vec{\delta^{(k)}} =
  \left\{
    \begin{array}{lcc}
      &(y - ŷ) &\odot f'_a(S^(k))                              & si & k = L \\
      &\vec{\delta^{(k+1)}} \cdot W^{(k+1)T} &\odot f'_a(S^(k)) & si & k < L
    \end{array}
  \right.
$$

Tras calcular los errores de todas las capas, las matrices de cambio de pesos se definen de forma muy parecida a la regla delta, sólo que con los nuevos errores:

$$
\Delta W^{(k)} = \alpha S^{(k-1)T} \delta^{(k)}
$$

Con todos los $\Delta W^{(k)}$ calculados podemos actualizar los pesos de cada capa $k$ de la red. De esta manera habríamos realizdo un paso de aprendizaje en nuestro proceso de entrenamiento.

Ahora implementaremos tanto el algoritmo de aprendizaje como el proceso de entrenamiento en nuestro perceptrón, ya que este no cambia: al ser un esquema de aprendizaje supervisado, el principio es el mismo. Además, implementaremos un nuevo indicador del error, el RMSE, ya que al ser la función de activación una sigmoidal, jamás llegará a los valores 0 o 1.

In [ ]:
class MultilayerPerceptron:
    def __init__(self, n_inputs, n_hidden, n_outputs):
        # Matriz de pesos capa 1: (filas: entradas, columnas: salidas)
        self.W1 = np.random.uniform(-0.5, 0.5, (n_inputs, n_hidden))
        self.b1 = np.random.uniform(-0.5, 0.5, (1, n_hidden))
        # Matriz de pesos capa 2: (filas: entradas, columnas: salidas)
        self.W2 = np.random.uniform(-0.5, 0.5, (n_hidden, n_outputs))
        self.b2 = np.random.uniform(-0.5, 0.5, (1, n_outputs))

    def activation(self, X):
        # Función de activación: función sigmoidal
        return 1 / (1 + np.exp(-X))
    
    def d_activation(self, X):
        # Derivada de la función de activación
        return X * (1 - X)
        
    def inference(self, X):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # Calculamos la salida de la capa 1
        self.S1 = self.activation(X @ self.W1 + self.b1)
        # Calculamos la salida de la capa 2
        self.S2 = self.activation(self.S1 @ self.W2 + self.b2)
        return self.S2
    
    def learn(self, X, y, alpha):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # Calculamos cuál es la salida para la entrada suministrada
        ŷ = self.inference(X)
        # Con ella, calculamos el error y la matriz de actualización de la última capa
        d2 = (y - ŷ) * self.d_activation(ŷ)
        dW2 = alpha * self.S1.T @ d2
        db2 = alpha * np.ones((1, d2.shape[0])) @ d2
        # Lo mismo pero con la primera capa
        d1 = d2 @ self.W2.T * self.d_activation(self.S1)
        dW1 = alpha * X.T @ d1
        db1 = alpha * np.ones((1, d1.shape[0])) @ d1
        # Por último, actualizamos los pesos de nuestra red
        self.W1 = self.W1 + dW1
        self.b1 = self.b1 + db1
        self.W2 = self.W2 + dW2
        self.b2 = self.b2 + db2
    
    def train(self, X, y, epochs, alpha, trace=1):
        for epoch in range(0, epochs):
            if epoch % trace == 0:
                accuracy, error = self.measures(X, y)
                print(f'Epoch {epoch}: Accuracy: {accuracy}, RMSE: {error}')
            self.learn(X, y, alpha)
        accuracy, error = self.measures(X, y)
        print(f'End -> {epoch}: Accuracy: {accuracy}, RMSE: {error}')
    
    def measures(self, X, y):
        ŷ = self.inference(X)
        accuracy = (y == ŷ).mean()
        rmse = np.sqrt(np.mean((y - ŷ)**2))
        return accuracy, rmse

Hay un detalle a la hora de actualizar los bias. Sólo hay un bias por neurona, por lo que si trabajamos con conjuntos de datos en lugar de ejemplos sueltos, hay que tenerlo en cuenta. Por eso se multiplica por un vector de 1s de la misma longitud que la de neuronas.

Ahora vamos a probar a entrenar nuestro perceptrón multicapa para una puerta OR de tres entradas 

In [ ]:
DATASET_AND = np.array([
    [0, 0, 0, 0],
    [0, 0, 1, 1],
    [0, 1, 0, 1],
    [0, 1, 1, 1],
    [1, 0, 0, 1],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [1, 1, 1, 1],
])
X = DATASET_AND[:, :-1]  # Entradas: Todas las columnas hasta la última
y = DATASET_AND[:, -1:]  # Salidas: Todas las columnas desde la última

model = MultilayerPerceptron(n_inputs=3, n_hidden=2, n_outputs=1)
model.train(X, y, 1000, 0.5, 100)

Veamos qué tal predice nuestro conjunto de datos de la puerta AND tras el entrenamiento

In [ ]:
ŷ = model.inference(X)
np.piecewise(ŷ, [ŷ < 0.5, 0.5 <= ŷ], [0, 1])

Bueno, parece que aprende. Vamos a ver qué tal lo hace con la puerta XOR

In [ ]:
DATASET_XOR = np.array([
    [0, 0, 0],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0],
])
X = DATASET_XOR[:, :-1]  # Entradas: Todas las columnas hasta la última
y = DATASET_XOR[:, -1:]  # Salidas: Todas las columnas desde la última

model = MultilayerPerceptron(n_inputs=2, n_hidden=3, n_outputs=1)
model.train(X, y, 1000, 0.5, 100)
ŷ = model.inference(X)
np.piecewise(ŷ, [ŷ < 0.5, 0.5 <= ŷ], [0, 1])

¡Fantástico! ¡Parece que lo ha aprendido! Acabamos de resolver el problema que causó que durante una decada ni se hablase de redes neuronales. Hablamos de uno de los múltiples [Inviernos de la IA](https://en.wikipedia.org/wiki/AI_winter). En este en concreto, se abandonó durante algo más de una década la investigación en redes neuronales debido a las limitaciones del perceptrón.

Ahora vamos a continuar con nuestro perceptrón multicapa

### MLP con varias capas ocultas

Una pequeña vuelta de tuerca a nuestra implementación. Con una única capa oculta nos basta para aproximar cualquier función (son [aproximadores universales](https://en.wikipedia.org/wiki/Universal_approximation_theorem) de funciones), pero no son suficientes para generalizar. De hecho, más capas ocultas con menos neuronas en total suelen resolver mejor los problemas gracias a su capacidad de generalización.

Por tanto, reimplementaremos el perceptrón, esta vez para que admita un número indefinido de capas y neuronas. Concretamente, en el constructor se recibirá una lista de enteros cuyo primer valor será el número de valores de entrada, su último valor el número de neuronas de salida y los valores intermedios el número de neuronas de cada una de las capas. Por ejemplo, `layers = [2, 3, 4, 1]` se correspondería con 2 neuronas de entrada, una capa oculta de 3 neuronas, otra capa oculta de 4 neuronas y una capa de salida de una neurona.

In [ ]:
class MultilayerPerceptron:
    def __init__(self, layers):
        # Los pesos de cada capa. Añadimos un elemento vacío al principio para que
        # los índices se correspondan con los de las funciones.
        self.W = [None] + [
            np.random.uniform(-0.5, 0.5, (prev_neurons, curr_neurons))
            for prev_neurons, curr_neurons in zip(layers[:-1], layers[1:])
        ]
        # Los bias de cada capa. Lo mismo que antes con el elemento vacío al ppio.
        self.b = [None] + [
            np.random.uniform(-0.5, 0.5, (1, curr_neurons))
            for curr_neurons in layers[1:]
        ]
        # La caché de salidas intermedias
        self.S = [None for _ in layers]

    def activation(self, X):
        # Función de activación: función sigmoidal
        return 1 / (1 + np.exp(-X))
    
    def d_activation(self, X):
        # Derivada de la función de activación
        return X * (1 - X)
        
    def inference(self, X):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # La salida 0 será la entrada a la red
        self.S[0] = X
        # El resto de salidas se calculan a partir de la salida anterior
        for i in range(1, len(self.S)):
            self.S[i] = self.activation(self.S[i-1] @ self.W[i] + self.b[i])
        # La última de nuestras salidas es la salida de la red
        return self.S[-1]
    
    def learn(self, X, y, alpha):
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis=0)
        # Calculamos todas las salidas de nuestra red
        ŷ = self.inference(X)
        # Con ellas, vamos calculando los sucesivos errores y matrices de
        # actualización. Comenzamos por la última capa:
        δ = (y - ŷ) * self.d_activation(ŷ)
        δW = [alpha * self.S[-2].T @ δ]
        δb = [alpha * np.ones((1, δ.shape[0])) @ δ]
        # Seguimos por las capas intermedias hasta el principio de la red
        for i in range(len(self.S) - 2, 0, -1):
            δ = δ @ self.W[i+1].T * self.d_activation(self.S[i])
            δW.append(alpha * self.S[i-1].T @ δ)
            δb.append(alpha * np.ones((1, δ.shape[0])) @ δ)
        # Por último, actualizamos los pesos de nuestra red
        for i, (δW, δb) in enumerate(zip(reversed(δW), reversed(δb)), 1):
            self.W[i] = self.W[i] + δW
            self.b[i] = self.b[i] + δb
    
    def train(self, X, y, epochs, alpha, trace=1):
        for epoch in range(1, epochs):
            if epoch % trace == 0:
                accuracy, error = self.measures(X, y)
                print(f'Epoch {epoch}: Accuracy: {accuracy}, RMSE: {error}')
            self.learn(X, y, alpha)
        accuracy, error = self.measures(X, y)
        print(f'End -> {epoch}: Accuracy: {accuracy}, RMSE: {error}')
    
    def measures(self, X, y):
        ŷ = self.inference(X)
        accuracy = (y == ŷ).mean()
        rmse = np.sqrt(np.mean((y - ŷ)**2))
        return accuracy, rmse


DATASET_XOR = np.array([
    [0, 0, 0],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0],
])
X = DATASET_XOR[:, :-1]  # Entradas: Todas las columnas hasta la última
y = DATASET_XOR[:, -1:]  # Salidas: Todas las columnas desde la última

model = MultilayerPerceptron([2, 2, 2, 1])
model.train(X, y, 10000, 0.5, 1000)
ŷ = model.inference(X)
np.piecewise(ŷ, [ŷ < 0.5, 0.5 <= ŷ], [0, 1])

El problema del XOR tiene bastantes mínimos locales que hacen que en ocasiones el entrenamiento se quede estancado en un punto concreto. Por ello, probablemente se necesiten varias ejecuciones del entrenamiento para tener un modelo que sea capaz de clasificar esta puerta

## Implementación con Keras

Hasta ahora hemos implementado desde cero un perceptrón y un perceptrón multicapa. Sin embargo, existen bibliotecas de alto nivel que nos permiten crear y entrenar redes neuronales de manera más eficiente y sencilla. Una de las bibliotecas más populares para este propósito es Keras, una API de redes neuronales de código abierto que se ejecuta sobre TensorFlow, Theano o CNTK.

### Conjunto de datos sobre el que trabajar

Ahora estamos trabajando con modelos que aprenden bajo un esquema de aprendizaje supervisado por lo que necesitamos un conjunto de datos con entradas y sus salidas esperadas de los que aprender.

Keras nos proporciona varios conjuntos de datos bajo el módulo `datasets`. De ellos vamos a usar el de `mnist`, que es un conjunto de 70000 imágenes (60000 en el conjunto de entrenamiento, 10000 en el conjunto de test) de dígitos escritos a mano de $28 \times 28$ píxeles junto con su correspondiente etiqueta indicando qué número es exactamente.

<center>
<figure class="image">
    <img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png" alt="Conjunto de datos MNIST" />
    <figcaption><em><strong>Figura 1.</strong>Ilustración de imágenes de ejemplo del conjunto de datos MNIST. Fuente: [Wikipedia](https://es.wikipedia.org/wiki/Base_de_datos_MNIST).</em></figcaption>
</figure>
</center>

Los _datasets_ del módulo ofrecen una función `load_data` para cargar los datos, descargándolos si es necesario. En el caso del `mnist`, los datos vienen separados en dos conjuntos, entrenamiento y test, en dos partes cada uno, entradas y salidas.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print(f'Training shape: {x_train.shape} input, {y_train.shape} output')
print(f'Test shape:     {x_test.shape} input, {y_test.shape} output')

Con los conjuntos de datos cargados y preparados, ya tenemos suficiente para trabajar. ¡Vamos allá!

### El API secuencial

La primera forma que vamos a ver para crear modelos en `keras` es su API secuencial. Es la más sencilla, ya que el modelo se crea como una lista de capas sucesivas y por debajo este las conecta (esto es, las salidas de una capa con las entradas de la siguiente). Por ejemplo, el perceptrón que implementamos anteriormente (bueno, lo más parecido a lo que podemos llegar) se implementaría como sigue:

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(3, activation='sigmoid', input_shape = [2,]),
    tf.keras.layers.Dense(2, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
model.summary()

Esto sería únicamente la arquitectura. Para poder entrenarlo tendríamos que especificar de qué manera calculamos el error (en nuestro caso antes usábamos la diferencia entre valor esperado y valor inferido), qué optimizador (en nuestro caso usábamos _backpropagation_) y si queremos o no alguna métrica más (antes sacábamos exactitud y rmse).

In [ ]:
model.compile(
    loss = tf.keras.losses.MeanSquaredError(),
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.5),
    metrics = [tf.keras.metrics.Accuracy()]
)

Con esto podemos entrenar el modelo. Por ejemplo, en el problema de la puerta XOR de antes:

In [ ]:
DATASET_XOR = np.array([
    [0, 0, 0],
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0],
])
X = DATASET_XOR[:, :-1]  # Entradas: Todas las columnas hasta la última
y = DATASET_XOR[:, -1:]  # Salidas: Todas las columnas desde la última

model.fit(X, y, epochs=1000, verbose=0)
ŷ = model.predict(X)
np.piecewise(ŷ, [ŷ < 0.5, 0.5 <= ŷ], [0, 1])

Ahora bien, con el conjunto de datos de `mnist` tenemos que hacer algunos cambios porque este modelo no nos vale:

1. La entrada es una matriz de $28 \times 28$, y hasta ahora hemos visto modelos que esperan un vector de entrada. Afortunadamente para esto último `keras` proporciona una capa denominada `Flatten`, que simplifica nuestros datos eliminando la información sobre la estructura 2D de la imagen.
2. La salida de nuestro modelo no es 0 o 1. Es un valor del 0 al 10. Si fuese una clasificación binaria valdría, pero es una clasificación denominada "multiclase". Para estos casos, se suele trabajar con varias clasificaciones binarias, una para cada clase. Como tenemos 10 posibles valores de salida, usaremos 10 neuronas, una para cada clase, y como función de activación y loss usaremos dos funciones que trabajan juntas para hacer las labores de clasificación. Esto lo veremos más adelante; por ahora basta pensar que funcionan juntas en este tipo de problemas multiclase.
3. El learning_rate que se suele usar para empezar a entrenar modelos suele ser bajo, del orden de 0.1 o inferior. Usaremos los valroes por defecto de nuestro optimizador.
4. Existe una métrica más adecuada para nuestro cálculo del error que la simple exactitud.

Con estos cambios nuestro modelo quedaría como sigue:

In [ ]:
model = tf.keras.models.Sequential([
    # La primera capa del modelo toma entradas de 28x28 y las "aplana"
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(3, activation='sigmoid'),
    tf.keras.layers.Dense(2, activation='sigmoid'),
    # La salida la cambiamos a 10 neuronas y una función de activación softmax
    tf.keras.layers.Dense(10, activation='softmax'),
])
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.SGD(),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
)
model.summary()

Es curioso ver cómo han cambiado los parámetros. De 2 entradas que teníamos antes a 768, con lo que ahora tenemos que ajustar 2393 parámetros en lugar de sólo 20. Bueno, veamos qué tal se comporta con el conjunto de entrenamiento de `mnist`

In [ ]:
history = model.fit(x_train, y_train, epochs=10)

Como vemos, el método `fit` devuelve un objeto al que apuntará la variable `history`. Este objeto guarda un histórico de todos los indicadores de nuestro proceso de entrenamiento (incluidas las métricas especificadas en el método `compile`) _epoch_ por _epoch_. Podemos aprovechar este objeto para imprimir por pantalla la evolución del entrenamiento, lo que nos daría información acerca de cómo ha ido:

In [ ]:
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

Esta gráfica nos dice más o menos tres cosas:

1. El error (_loss_) va bajando durante el entrenamiento, cosa que está muy bien.
2. La exactitud (_accuracy_) va subiendo, cosa que también está muy bien.
3. La exactitud sigue siendo baja, por lo que queda mucho que entrenar.

Vamos a ver la evolución de los indicadores con unos cuantos _epochs_ más de entrenamiento.

In [ ]:
history = model.fit(x_train, y_train, epochs=100)
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

Hemos realizado 100 epochs a partir del modelo ya entrenado y parece que, aunque sigue evolucionando, su capacidad predictiva deja un poco que desear. Vamos a entrenar un modelo un poco más complejo a ver si somos capaces de que la precisión aumente.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(5, activation='sigmoid'),
    tf.keras.layers.Dense(5, activation='sigmoid'),
    tf.keras.layers.Dense(10, activation='softmax'),
])
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.SGD(),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
)
model.summary()

Para el entrenamiento vamos a hacer uso de un argumento más, `validation_split=0.1`. Con este argumento el conjunto de entrenamiento se separará en 2 conjuntos diferentes durante cada _epoch_: uno con el 90% de los datos, con el que se realizará el entrenamiento y otro con el 10% de datos restante para evaluar la red tras el entrenamiento de ese epoch. Vamos allá:

In [ ]:
history = model.fit(x_train, y_train, epochs=100, validation_split=0.1)

Con este argumento podemos ser capaces de tener una intuición no solo de cómo va aprendiendo la red, sino de su evolución en la capacidad de predicción.

In [ ]:
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

Un entrenamiento que tiene mejor pinta. parece que más o menos el modelo acierta un 77% de los casos en entrenamiento y un 74% de los casos en validación. No es una diferencia muy grande, así que podemos decir que no está sobreespecializado (sufriendo _overfitting_).

Si damos como bueno este modelo, el siguiente (y último paso) para determinar si es apto para usarse en el mundo real sería comprobar con el conjunto de test. Veamos como se comporta con este:

In [ ]:
model.evaluate(x_test, y_test)

Aproximadamente un 75% de exactitud, no está mal. O no. Todo depende del problema, ya que un 75% de aciertos en reconocer un cáncer de mama no es una estadística muy halagüeña. Pero no nos preocupemos, hay más modelos que aprenderemos para abordar diferentes casos. Por ahora, estamos aprendiendo las diferentes formas de aprender modelos con Keras. Vamos a por la segunda forma.

### El API funcional

Muchos modelos de aprendizaje profundo son grafos acíclicos dirigidos (DAG, del inglés _directed acyclic graph_) donde cada nodo se corresponde con una capa. El API funcional nos permite construirlos en términos de entradas y salidas de capas.

Esto es debido a que, en el momento que queremos interconectar las capas de una manera diferente (por ejemplo una entrada común que va a dos subredes independientes que luego convergen en una capa común), el API secuencial no nos sirve.

Nos encontraremos con redes complicadas de este tipo más adelante. Por ahora nos ceñiremos a nuestro perceptrón multicapa y su definición con este API. Veamos cómo sería:

In [ ]:
# Definimos las capas con sus conexiones explícitamente
input_layer = tf.keras.layers.Input(shape=(28, 28))
flatten = tf.keras.layers.Flatten()(input_layer)
hidden_1 = tf.keras.layers.Dense(5, activation='sigmoid')(flatten)
hidden_2 = tf.keras.layers.Dense(5, activation='sigmoid')(hidden_1)
output_layer = tf.keras.layers.Dense(10, activation='softmax')(hidden_2)
# Creamos el modelo especificando cuáles son las entradas y cuáles las salidas
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
# La arquitectura ya está definida, así que ya podemos compilarlo
model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.SGD(),
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
)
model.summary()

Exactamente los mismos parámetros y la misma arquitectura, sólo que definido de manera diferente. Ahora vamos a realizar un nuevo entrenamiento para ver si la tendencia es similar a la anterior (que esperemos que lo sea):

In [ ]:
history = model.fit(x_train, y_train, epochs=100, validation_split=0.1)

Desde luego por los valores de los indicadores en entrenamiento y en validación, parece que se está comportando de la misma manera. Veamos la tendencia en una gráfica, que siempre es más agradecida.

In [ ]:
pd.DataFrame(history.history).plot()
plt.xlabel('Epoch num.')
plt.show()

En efecto, se parecen bastante. Y menos mal, porque si no todo esto no valdría para nada. Ya conocemos las dos formas que tiene Keras para definir modelos de redes neuronales.

***

## Conclusiones

Hemos cubierto un espectro amplio y fundamental del aprendizaje automático y el aprendizaje profundo. Cada uno de estos modelos nos ha ofrecido una perspectiva única sobre la resolución de problemas mediante algoritmos de inteligencia artificial, destacando tanto sus fortalezas como sus limitaciones.

Con el **perceptrón simple**, hemos establecido las bases del aprendizaje automático, comprendiendo la importancia de la linealidad y sus limitaciones inherentes al enfrentarnos a problemas más complejos y no lineales. Este modelo nos sirve como un recordatorio de que, aunque algunos problemas pueden ser abordados con soluciones simples y elegantes, el mundo real a menudo demanda enfoques más sofisticados.

Avanzando hacia el perceptrón multicapa y el aprendizaje profundo, exploramos la capacidad de estas redes para modelar complejidades mucho mayores gracias a sus capas ocultas y funciones de activación no lineales. Sin embargo, también observamos la creciente complejidad en su implementación y la necesidad de considerar aspectos críticos como la elección de funciones de activación adecuadas, la inicialización de pesos y técnicas para mejorar el entrenamiento y evitar el sobreajuste.

La introducción de Keras representa un hito, facilitando la implementación de perceptrones multicapa y abriendo la puerta a experimentar con arquitecturas más avanzadas de manera intuitiva y eficiente. Keras no solo simplifica el proceso de construcción y entrenamiento de modelos sino que también democratiza el acceso a tecnologías de aprendizaje profundo, permitiendo a investigadores y desarrolladores concentrarse en la resolución de problemas sin preocuparse excesivamente por los detalles de bajo nivel.

Finalmente, al considerar las posibilidades que ofrece la API de Subclassing de Keras, reconocemos la flexibilidad y el poder que se nos otorga para diseñar modelos que se salen del paradigma tradicional, abordando problemas que requieren estructuras dinámicas y personalizadas. Esto subraya la importancia de entender los fundamentos detrás de cada herramienta y modelo, ya que solo así podemos expandir creativamente sus límites.

***

## Referencias

[1] Guía para la creación de capas y modelos personalizados (<https://www.tensorflow.org/guide/keras/custom_layers_and_models>)

***

<div><img style="float: right; width: 120px; vertical-align:top" src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by-nc-sa.png" alt="Creative Commons by-nc-sa logo" />

[Volver al inicio](#top)

</div>